In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Define the path to your dataset directory
dataset_dir = r'C:\Users\sadia_tisha1\Downloads\archive'

# Initialize empty lists for X_train and Y_train
X_train = []
Y_train = []



In [2]:
# Iterate through each subfolder in the dataset directory
for label, class_folder in enumerate(os.listdir(dataset_dir)):
    class_folder_path = os.path.join(dataset_dir, class_folder)

    # Check if it's a directory
    if os.path.isdir(class_folder_path):
        for image_file in os.listdir(class_folder_path):
            image_path = os.path.join(class_folder_path, image_file)
            
            # Open the image using PIL and convert to RGB
            image = Image.open(image_path).convert("RGB")
            
            # Resize the image to (32, 32, 3)
            image = image.resize((224, 224))
            
            # Convert PIL image to numpy array
            image_array = np.array(image)
            
            # Append the image data to X_train
            X_train.append(image_array)
            
            # Append the label to Y_train
            Y_train.append(label)

# Convert X_train and Y_train to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Convert Y_train and Y_test to categorical one-hot encoding
num_classes = len(np.unique(Y_train))
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)
Y_test_categorical = to_categorical(Y_test, num_classes=num_classes)

# Print the shapes of the arrays
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train_categorical.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test_categorical.shape)



X_train shape: (10968, 224, 224, 3)
Y_train shape: (10968, 23)
X_test shape: (2743, 224, 224, 3)
Y_test shape: (2743, 23)


In [3]:
# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Split your data into training and validation sets
X_train, X_valid, Y_train_categorical, Y_valid_categorical = train_test_split(
    X_train, Y_train_categorical, test_size=0.1, random_state=42
)

# Create tf.data.Dataset for training and validation data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train_categorical))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, Y_valid_categorical))

# Define batch size and shuffle the datasets
batch_size = 64
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
valid_dataset = valid_dataset.batch(batch_size)



In [4]:
# Training the model
epochs = 50
model.fit(train_dataset, epochs=epochs, validation_data=valid_dataset)


Epoch 1/50
155/155 [==============================] - 2684s 17s/step - loss: 1.6408 - accuracy: 0.5240 - val_loss: 5.8461 - val_accuracy: 0.2452
Epoch 2/50
155/155 [==============================] - 2672s 17s/step - loss: 1.0248 - accuracy: 0.6856 - val_loss: 1.7653 - val_accuracy: 0.4831
Epoch 3/50
155/155 [==============================] - 2691s 17s/step - loss: 0.6596 - accuracy: 0.7978 - val_loss: 2.7041 - val_accuracy: 0.4366
Epoch 4/50
155/155 [==============================] - 2695s 17s/step - loss: 0.4816 - accuracy: 0.8521 - val_loss: 1.7488 - val_accuracy: 0.5324
Epoch 5/50
155/155 [==============================] - 2671s 17s/step - loss: 0.3898 - accuracy: 0.8805 - val_loss: 6.3865 - val_accuracy: 0.2799
Epoch 6/50
155/155 [==============================] - 2672s 17s/step - loss: 0.2372 - accuracy: 0.9272 - val_loss: 1.6797 - val_accuracy: 0.5953
Epoch 7/50
155/155 [==============================] - 2671s 17s/step - loss: 0.1709 - accuracy: 0.9463 - val_loss: 2.0230 - val_ac

In [6]:

from sklearn.metrics import accuracy_score, f1_score

# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test_categorical))
test_dataset = test_dataset.batch(batch_size)

# Use the model to make predictions on the test dataset
y_pred = model.predict(test_dataset)

# Convert one-hot encoded predictions back to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred_labels)

# Calculate F1 score
f1 = f1_score(Y_test, y_pred_labels, average='weighted')  # You can specify the averaging method

# Print accuracy and F1 score
print("Accuracy:", accuracy)
print("F1 Score:", f1)

Accuracy: 0.7243893547211083
F1 Score: 0.721512426207462
